# Predicting Diabetes using Artificial Neural Network

In this notebook we will learn how to predict whether a person will acquire diabetes in her lifetime given various body parameters using Neural Networks

## Importing Dependencies
1. [Numpy](https://www.numpy.org/): For matrix calculations
2. [Pandas](https://pandas.pydata.org/): For reading csv file
3. [Matplotlib](https://matplotlib.org/): For plotting data
4. [Scikit-Learn](https://scikit-learn.org/): Splitting and testing data

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm_notebook
import h5py
%matplotlib inline

## for producing consistent results
tf.set_random_seed(4)

## Loading Dataset
Dataset contains following columns:
1. Pregnancies: Number of times pregnant
2. Glucose: Plasma glucose concentration a 2 hours in an oral glucose tolerance test
3. BloodPressure: Diastolic blood pressure (mm Hg)
4. SkinThickness: Triceps skin fold thickness (mm)
5. Insulin: 2-Hour serum insulin (mu U/ml)
6. BMI: Body mass index (weight in kg/(height in m)^2)
7. DiabetesPedigreeFunction: Diabetes pedigree function
8. Age: Age (years)
9. Outcome:  Class variable (0 or 1)

More inofrmation about dataset can be found [here](https://www.kaggle.com/uciml/pima-indians-diabetes-database)

In [ ]:
## dataset link
DATA_LINK = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

## reading csv file present at url
data = pd.read_csv(DATA_LINK, header = None)

## first 10 rows
data.head(10)

In [ ]:
## extracting features
X_data = np.array(data.iloc[:, 0:8])

## normalizing dataset
X_data -= np.mean(X_data, axis = 0)
X_data /= np.std(X_data, axis = 0)

## target
y_data = np.array(data.iloc[:, 8])

## the numpy array returned is of shape (n_examples,) but we need a 2-D array to feed to tensor
y_data = np.expand_dims(y_data, axis = 1)

## Splitting Dataset
For the purpose of training and testing, we split the dataset in ratio of 70% (training) and 30% (testing) <br/>
For this purpose we use [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function of [scikit-learn](https://scikit-learn.org/)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state = 4)

In [ ]:
## placeholder for features
X = tf.placeholder(tf.float32, [None, 8])

## placeholder for output
y = tf.placeholder(tf.float32, [None, 1])

## Neural Network Architecture
In this problem we use a Dense Neural Network, with 2 hidden layers with 12 and 8 hidden units, respectively. <br/>
For the first hidden layer, relu activation is used and for the second hidden unit, tanh activation is used.
<img src = "Images/nn.svg">

In [ ]:
## weights for the model
weights = {'W1': tf.Variable(np.random.randn(8, 12), dtype = tf.float32),
           'b1': tf.Variable(np.random.randn(12), dtype = tf.float32),
           'W2': tf.Variable(np.random.randn(12, 8), dtype = tf.float32),
           'b2': tf.Variable(np.random.randn(8), dtype = tf.float32),
           'W3': tf.Variable(np.random.randn(8, 1), dtype = tf.float32),
           'b3': tf.Variable(np.random.randn(1), dtype = tf.float32)
          }

In [ ]:
def generateProbability(weights, X):
    '''
    
    '''
    
    ## first layer
    layer_1 = tf.matmul(X, weights["W1"]) + weights["b1"]
    ## relu activation
    layer_1 = tf.nn.relu(layer_1)
    
    ## second layer
    layer_2 = tf.matmul(layer_1, weights["W2"]) + weights["b2"]
    ## tanh activation
    layer_2 = tf.nn.tanh(layer_2)
    
    ## third layer
    layer_3 = tf.matmul(layer_2,weights["W3"]) + weights["b3"]
    return tf.sigmoid(layer_3)

In [ ]:
def loss(weights, X, y):
    predict_prob = generateProbability(weights, X)
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = predict_prob, labels = y))

In [ ]:
def predict(weights, X):
    predict_proba = generateProbability(weights, X)
    return tf.math.floor(2 * predict_proba)

In [ ]:
cost = loss(weights, X, y)
pred = predict(weights, X)
optimizer = tf.train.AdamOptimizer(0.01).minimize(cost)
init = tf.global_variables_initializer()

In [ ]:
%%time
train_cost_cache = []
test_cost_cache = []
train_acc_cache = []
test_acc_cache = []

with tf.Session() as sess:
    ## initializing global variables
    sess.run(init)
    for num_iter in tqdm_notebook(range(250)):
        
        ## data fed through feed dictionary
        ## returns the optimizer and current cost
        _, train_cost, train_predictions = sess.run([optimizer, cost, pred], feed_dict = {X: X_train, y: y_train})
        test_cost, test_predictions = sess.run([cost, pred], feed_dict = {X: X_test, y: y_test})
        
        
        if num_iter % 10 == 0:
            train_cost_cache.append(train_cost)
            
            print("Iteration: {}, Training Loss = {}".format(num_iter, train_cost))
            print("Training Accuracy = {}".format(accuracy_score(train_predictions, y_train)))
            
            train_acc_cache.append(accuracy_score(train_predictions, y_train))
            
            test_cost_cache.append(test_cost)
            
            print("Test Loss = {}".format(test_cost))
            print("Test Accuracy = {}".format(accuracy_score(test_predictions, y_test)))
            test_acc_cache.append(accuracy_score(test_predictions, y_test))

In [ ]:
fig = plt.figure(figsize = (10, 10))

plt.subplot(2, 2, 1)
plt.plot(train_cost_cache)
plt.ylabel('Training Loss')

plt.subplot(2, 2, 2)
plt.plot(train_acc_cache)
plt.ylabel('Training Accuracy')

plt.subplot(2, 2, 3)
plt.plot(test_cost_cache)
plt.ylabel('Test loss')

plt.subplot(2, 2, 4)
plt.plot(test_acc_cache)
plt.ylabel('Test Accuracy')


## Using Keras

In [ ]:
from tensorflow.keras.models import Sequential, load_model, save_model
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=150, batch_size=10, validation_data=(X_test, y_test))

In [ ]:
model.save("diabetes.h5")

In [ ]:
del model
model = load_model("diabetes.h5")